# Training and testing of objection detection network
Based on Faster-RCNN and this guide: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html


In [1]:
# %%shell

# pip install cython
# # Install pycocotools, the version by default in Colab
# # has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
# pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

## Importing libraries and custom scripts


In [2]:
import sys
import os
import numpy as np
import torch
import torch.utils.data

sys.path.append(os.getcwd() + "/.." + "/scripts")


## Dataset class with new __getitem__ function

In [3]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import transforms as T






class CansDataset(torch.utils.data.Dataset):
    def __init__(self, root, train=True):
        if train is True:
            self.root = root + "/video1/train/"
        else:
            self.root = root +  "/video1/test/"

        self.imgs = list(sorted(os.listdir(os.path.join(self.root,"frames"))))
        self.bbox = list(sorted(os.listdir(os.path.join(self.root,"boundingboxes"))))
    
    def __getitem__(self, idx):
        # load images and bboxes
        img_path = os.path.join(self.root, "frames", self.imgs[idx])
        bbox_path = os.path.join(self.root, "boundingboxes", self.bbox[idx])
        img = Image.open(img_path).convert("RGB")
        img=np.array(img)
        img=torch.tensor(img)/255
        img=img.permute(2,0,1)

        bbox = []
        label = []
        with open(bbox_path, 'r') as f:
            for line in f:
                line = line.split(" ")
                id = line[0] # class label, 1=beer, 2=cola, 0=background
                id = 1 if id == 'beer' else 2
                xmin = float(line[1])
                ymin = float(line[2])
                xmax = float(line[3])
                ymax = float(line[4])
                bbox.append([xmin, ymin, xmax, ymax])
                label.append(id)
        bbox = torch.as_tensor(bbox, dtype=torch.int64)
        labels = torch.as_tensor(label, dtype=torch.int64) #torch.ones((num_objs, ), dtype=torch.float32)
        image_id = torch.tensor([idx],dtype=torch.int64)
        area = (bbox[:, 3] - bbox[:, 1]) * (bbox[:, 2] - bbox[:, 0])

        target = {}
        target["boxes"] = bbox
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area


        return img, target

    def __len__(self):
        return len(self.imgs)

Test class

In [4]:

#root = os.getcwd()+'data/'#os.getcwd() + '/..' + '/data/'
sys.path.append(os.getcwd() + "/.." + "/scripts")
print(os.getcwd())
os.chdir(os.getcwd()+"/..")

#print(root)
root = os.getcwd() + "/data"


/home/andreasgp/MEGAsync/DTU/9. Semester/Deep Learning/object-tracking-project/02456-project/notebooks


## Adding pretrained model and modify numbers of classes

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 3  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [6]:
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [7]:

from engine import train_one_epoch, evaluate

import utils
import transforms as T
from typing import List, Tuple





dataset = CansDataset(root, train=True)
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)


print(images[0])

targets = [{k: v for k, v in t.items()} for t in targets]

output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions
print(predictions)


/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  

tensor([[[0.3804, 0.3882, 0.3647,  ..., 0.3647, 0.3843, 0.3922],
         [0.3843, 0.3490, 0.3294,  ..., 0.3725, 0.3922, 0.3961],
         [0.3765, 0.3373, 0.3529,  ..., 0.3725, 0.3922, 0.3961],
         ...,
         [0.1059, 0.1059, 0.1020,  ..., 0.0863, 0.0863, 0.0902],
         [0.1059, 0.1059, 0.1020,  ..., 0.0863, 0.0863, 0.0902],
         [0.1059, 0.1059, 0.1020,  ..., 0.0863, 0.0863, 0.0902]],

        [[0.3294, 0.3412, 0.3098,  ..., 0.3059, 0.3216, 0.3216],
         [0.3373, 0.3020, 0.2745,  ..., 0.3137, 0.3294, 0.3255],
         [0.3176, 0.2824, 0.2902,  ..., 0.3137, 0.3294, 0.3255],
         ...,
         [0.1098, 0.1098, 0.1059,  ..., 0.0902, 0.0863, 0.0902],
         [0.1098, 0.1098, 0.1059,  ..., 0.0902, 0.0863, 0.0902],
         [0.1098, 0.1098, 0.1059,  ..., 0.0902, 0.0863, 0.0902]],

        [[0.2549, 0.2549, 0.2078,  ..., 0.1843, 0.1882, 0.1804],
         [0.2510, 0.2078, 0.1725,  ..., 0.1922, 0.1961, 0.1843],
         [0.2275, 0.1804, 0.1882,  ..., 0.1922, 0.1961, 0.

/home/andreasgp/.local/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[{'boxes': tensor([[ 48.9748,  76.4988,  52.4558,  80.8412],
        [216.0179, 169.6655, 219.3163, 173.5281],
        [163.2359, 202.8737, 166.5454, 206.4323],
        [ 48.9596,  74.9428,  52.4041,  79.3189],
        [365.5294, 160.9054, 369.8692, 165.2826],
        [ 47.5540,  76.4168,  51.4282,  80.9287],
        [ 46.3876,  72.6617,  50.0949,  77.8197],
        [ 47.4495,  74.5400,  51.2161,  79.0829],
        [217.4637, 170.2014, 220.6037, 173.8633],
        [ 52.1910,  75.3912,  55.7675,  79.7023],
        [323.9361, 103.7627, 327.5754, 107.9266],
        [ 52.1118,  73.7258,  55.7016,  77.9808],
        [ 50.9825,  73.5484,  54.4292,  77.9100],
        [ 50.6271,  76.5666,  54.1327,  80.9554],
        [329.8467,  96.2168, 333.7085, 100.7062],
        [216.0104, 171.4740, 219.1762, 175.1339],
        [ 50.8987,  71.7445,  54.3731,  76.2274],
        [242.0988, 103.8065, 245.1979, 108.3394],
        [234.4603, 170.0946, 237.4371, 173.6755],
        [164.8206, 201.2403, 168.2403, 

### To Do
Fix get item så den kører med Jonas' data\
Få vores data til at kører med den pretrænede model\
Test forward pass\
Benyt Holgers split til at træne med og opnå fuld model



In [8]:
# use our dataset and defined transformations
dataset = CansDataset(root, train=True)
dataset_test = CansDataset(root, train=False)




# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has three classes only - background, beer and coke
num_classes = 3

# get the model using our helper function
#model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [10]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bbox = torch.as_tensor(bbox, dtype=torch.int64)
/tmp/ipykernel_64387/1320764867.py:45: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  

Epoch: [0]  [  0/590]  eta: 2:12:41  lr: 0.000013  loss: 1.5829 (1.5829)  loss_classifier: 1.2614 (1.2614)  loss_box_reg: 0.3137 (0.3137)  loss_objectness: 0.0068 (0.0068)  loss_rpn_box_reg: 0.0010 (0.0010)  time: 13.4944  data: 0.2846


In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])